In [1]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import pingouin as pg
import warnings

# **<font color = 'DarkRed'> County and District Levels Plots </font>**

* <font color='royalblue'> _Map_State displays the map of the congressional districts (if level = 'Congressional District'), counties (if level = 'County'), or health districts (if level = 'Health District') of a given state, colored by the data in the dataframe, df, in the data column given by data_col.<br> If level = 'Congressional District', then the dataframe, df, should have the congressional districts as index.<br> If level = 'County', then the dataframe, df, should have the county codes as index.<br> If level = 'Health District', then the dataframe, df, should have the health districts names as index.<br> The log_scale default is set to False. If it is set to True then log(data_col) values will be represented._ </font>
* <font color='green'> _Animated_Map_State is like Map_State with an animation slide added._ </font>

In [2]:
def Map_State(State_Code, level, df, data_col, Log_Scale = False):
    if level == 'Congressional District':
        geoDict = Get_geojson_CD116(State_Code)
        featureID = 'properties.CD116FP'
        hover_info = df.index.get_level_values(0)
    elif level == 'County':
        geoDict = Get_geojson_Counties(State_Code)
        featureID = 'properties.COUNTYFP'
        hover_info = [(Get_CountyNames_Dict(State_Code))[x] for x in df.index.get_level_values(0)]
    elif level == 'Health District':
        return
    else:
        print("The options for level should be: Congressional District, County, or Health District")
        return
    
    if Log_Scale:
        eps = 1e-10
        fig = px.choropleth(df, geojson = geoDict, 
                            locations = df.index.get_level_values(0), 
                            color = np.log(df[data_col] + eps),
                            hover_name = hover_info,
                            featureidkey = featureID,
                            color_continuous_scale = 'Portland', 
                            projection = 'natural earth', 
                            labels={'color' : 'Log ' + data_col})      
    else:
        fig = px.choropleth(df, geojson = geoDict, 
                            locations = df.index.get_level_values(0), 
                            color = data_col,
                            hover_name = hover_info,
                            featureidkey = featureID, 
                            color_continuous_scale = 'Portland', 
                            projection = 'natural earth')
    
    fig.update_geos(fitbounds = "locations", visible = False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show(renderer = 'browser')
    
    return

In [3]:
def Animated_Map_State(State_Code, level, df, data_col, animation_col, MyTitle, Log_Scale = False):
    if level == 'Congressional District':
        geoDict = Get_geojson_CD116(State_Code)
        featureID = 'properties.CD116FP'
        hover_info = df.index.get_level_values(0)
    elif level == 'County':
        geoDict = Get_geojson_Counties(State_Code)
        featureID = 'properties.COUNTYFP'
        hover_info = 'County Name'
    else:
        print("The options for level should be: Congressional District or County")
        return
          
    min_df = df[data_col].min()
    max_df = df[data_col].max()
    
    if Log_Scale:
        eps = 1e-10
        data = np.log(df[data_col] + eps)
        data_range = [np.log(min_df + eps), np.log(max_df + eps)]
        data_mean = (np.log(df[data_col] + eps)).mean()

        fig = px.choropleth(df, geojson = geoDict, 
                            locations = df.index.get_level_values(0), 
                            color = data,
                            hover_name = hover_info,
                            featureidkey = featureID, 
                            animation_frame = animation_col,
                            color_continuous_scale = px.colors.diverging.Spectral[::-1], 
                            color_continuous_midpoint = data_mean, 
                            range_color = data_range,
                            projection = 'natural earth', 
                            labels={'color' : 'Log ' + data_col}, 
                            fitbounds = 'locations', 
                            basemap_visible = False)     
    else:
        data_mean = df[data_col].mean()
        fig = px.choropleth(df, geojson = geoDict, 
                            locations = df.index.get_level_values(0), 
                            color = data_col, 
                            hover_name = hover_info,
                            featureidkey = featureID,
                            animation_frame = animation_col, 
                            color_continuous_scale = px.colors.diverging.Spectral[::-1],
                            color_continuous_midpoint = data_mean,
                            range_color = [min_df, max_df],
                            projection = 'natural earth', 
                            fitbounds = 'locations', 
                            basemap_visible = False)

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(title_text = MyTitle, title_x = 0.5, title_y = 0.99, title_yanchor = 'top',
                      title_font_size = 20, title_font_color = 'royalblue')

    fig.show(renderer = 'browser')

    return

In [4]:
def TimeSeries_Plot(State_Code, scope, fig_type, df, col, Log_Scale, My_Title):
    df['Report Date'] = pd.to_datetime(df['Report Date'], yearfirst = True)
    
    if scope == 'Congressional District':
        MyColors = px.colors.qualitative.Prism
        locations = df.index.get_level_values(0)
        hover_info = locations
    elif scope == 'County':
        MyColors = px.colors.qualitative.Alphabet
        locations = df['County Name']
        hover_info = df.index.get_level_values(0)        
    
    if fig_type == 'scatter':
        px_plot = px.scatter
    elif fig_type == 'line':
        px_plot = px.line
    else:
        px_plot = px.bar
        
    if Log_Scale:
        eps = 1e-3
        fig = px_plot(df, x = 'Report Date', y = np.log(df[col] + eps), color = locations, 
                      color_discrete_sequence = MyColors, hover_name = hover_info)
        fig.update_yaxes(title_text = 'Log ' + col)
    else:
        fig = px_plot(df, x = 'Report Date', y = col, color = locations,
                      color_discrete_sequence = MyColors, hover_name = hover_info)
        
    fig.update_xaxes(title_text = 'Report Date', 
                     tickmode = 'auto', nticks = 40,
                     tickangle = 315, tickformat = '%b %d %Y')
        
    fig.update_layout(title = My_Title, title_x = 0.5, legend_title = scope)
    
    if fig_type == 'bar':
        fig.update_layout(barmode = 'group')
        fig.update_traces(marker_line_width = 0, selector = dict(type = "bar"))
    
    fig.show(renderer = 'browser')
    return

In [5]:
def TimeSeries_Norm_Plots(State_Code, scope, fig_type, df, col, Log_Scale, My_Title):
    if scope == 'Congressional District':
        MyColors = px.colors.qualitative.Prism
        locations = df.index.get_level_values(0)
        hover_info = locations
    elif scope == 'County':
        MyColors = px.colors.qualitative.Alphabet
        locations = df['County Name']
        hover_info = df.index.get_level_values(0)
        
    locations_list = locations.unique().tolist()
    N = len(locations_list)
    df['Report Date'] = pd.to_datetime(df['Report Date'], yearfirst = True)
    
    if fig_type == 'scatter':
        px_plot = px.scatter
    elif fig_type == 'line':
        px_plot = px.line
    else:
        px_plot = px.bar   
    
    fig = make_subplots(rows = 1, cols = 2, horizontal_spacing = 0.08)
    
    if Log_Scale:
        eps = 1e-3
        y_data = np.log(df[col] + eps)
        fig.update_yaxes(title_text = 'Log ' + col, row = 1, col = 1)
    else:
        y_data = col
        fig.update_yaxes(title_text = col, row = 1, col = 1)
        
    trace = px_plot(df, x = 'Report Date', y = y_data, color = locations, 
                    color_discrete_sequence = MyColors, 
                    hover_name = hover_info)["data"] 
    
    for i in range(0, N):
        fig.add_trace(trace[i], row = 1, col = 1)
        
    y_data = col + ' minus State-Wide value'    
    trace = px_plot(df, x = 'Report Date', y = y_data, color = locations, 
                    color_discrete_sequence = MyColors,
                    hover_name = hover_info)["data"]
    fig.update_yaxes(title_text = y_data, row = 1, col = 2)
    
    for i in range(0, N):
        fig.add_trace(trace[i], row = 1, col = 2)
        
#     fig['data'][0]['showlegend'] = True
#     fig['data'][0]['name'] = 'State-Wide'
    
    fig.update_traces(marker_line_width = 0, selector = dict(type = "bar"))
    fig.update_traces(line_color='#000000', line_width = 4, selector = dict(name = 'State-Wide', type = 'scattergl'))
    fig.update_traces(marker_color = '#000000', selector = dict(name = 'State-Wide', type = 'bar'))
    
    if fig_type == 'scatter':
        fig.update_traces(marker_color = '#000000', marker_symbol = '302', selector = dict(name = 'State-Wide'))
    
    if fig_type == 'bar':
        fig.update_layout(barmode = 'group')
    
    for i in range(N, 2*N):
        fig["data"][i]['showlegend'] = False

    fig.update_xaxes(title_text = 'Report Date', 
                     tickmode = 'auto', nticks = 40,
                     tickangle = 315, tickformat = '%b %d %Y')
    
    fig.update_layout(title = My_Title, title_x = 0.5, legend_title = scope)
    
    fig.show(renderer = 'browser')
    return

In [6]:
def TimeSeries_Multi_Plots(State_Code, scope, fig_type, df, col, Log_Scale, My_Title):
    mask = df.index != 'State-Wide'
    if scope == 'Congressional District':
        MyColors = px.colors.qualitative.Prism
        locations = df[mask].index.get_level_values(0)
        hover_info = locations
    elif scope == 'County':
        MyColors = px.colors.qualitative.Alphabet
        locations = df[mask]['County Name']
        hover_info = df[mask].index.get_level_values(0)
        
    locations_list = locations.unique().tolist()
    N = len(locations_list)
    df['Report Date'] = pd.to_datetime(df['Report Date'], yearfirst = True)
    
    if fig_type == 'scatter':
        px_plot = px.scatter
    elif fig_type == 'line':
        px_plot = px.line
    else:
        px_plot = px.bar   
    
    fig = make_subplots(rows = 2, cols = 2, horizontal_spacing = 0.08, vertical_spacing = 0.03)
    
    if Log_Scale:
        eps = 1e-3
        y_data = np.log(df.loc['State-Wide', col] + eps)
        t11 = 'Log State-Wide ' + col
    else:
        y_data = col
        t11 = 'State-Wide ' + col
    
    trace = px_plot(df.loc['State-Wide'], x = 'Report Date', y = y_data, 
                    color_discrete_sequence = ['red'])["data"]
    
    fig.add_trace(trace[0], row = 1, col = 1)
    fig['data'][0]['showlegend']=True
    fig['data'][0]['name']='State-Wide'
    fig.update_xaxes(tickmode = 'auto', showticklabels = False, nticks = 40, row = 1, col = 1)
    fig.update_yaxes(title_text = t11, row = 1, col = 1)
    
    if Log_Scale:
        eps = 1e-3
        y_data = np.log(df[mask][col] + eps)
        t12 = 'Log ' + col
    else:
        y_data = col
        t12 = col
    
    trace = px_plot(df[mask], x = 'Report Date', y = y_data, color = locations,
                    color_discrete_sequence = MyColors, 
                    hover_name = hover_info)["data"]
    for i in range(0, N):
        fig.add_trace(trace[i], row = 1, col = 2)

    fig.update_xaxes(tickmode = 'auto', showticklabels = False, nticks = 40, row = 1, col = 2)
    fig.update_yaxes(title_text = t12, row = 1, col = 2)
    
    trace = px_plot(df[mask], x = 'Report Date', y = 'Diff', color = locations, 
                    color_discrete_sequence = MyColors, 
                    hover_name = hover_info)["data"]
    for i in range(0, N):
        fig.add_trace(trace[i], row = 2, col = 1)
        
    fig.update_xaxes(title_text = 'Report Date', tickmode = 'auto', tickangle = 315, nticks = 40, 
                     tickformat = '%b %d %Y', row = 2, col = 1)
    fig.update_yaxes(title_text = col + ' - State Average Value', row = 2, col = 1)
        
    trace = px_plot(df[mask], x = 'Report Date', y = 'Ratio', color = locations, 
                    color_discrete_sequence = MyColors, 
                    hover_name = hover_info)["data"]
    for i in range(0, N):
        fig.add_trace(trace[i], row = 2, col = 2)
        
    fig.update_xaxes(title_text = 'Report Date', tickmode = 'auto', tickangle = 315, nticks = 40, 
                     tickformat = '%b %d %Y', row = 2, col = 2)
    fig.update_yaxes(title_text = '(' + col + ' - State Average Value)/StateWide Value' , row = 2, col = 2)
    
    fig.update_traces(marker_line_width = 0, selector = dict(type = "bar"))
    fig.update_traces(line_color='#000000', line_width = 4, selector = dict(name = 'State Average', type = 'scattergl'))
    fig.update_traces(marker_color = '#000000', selector = dict(name = 'State Average', type = 'bar'))
    
    if fig_type == 'scatter':
        fig.update_traces(marker_color = '#000000', marker_symbol = '302', selector = dict(name = 'State Average'))
    
    if fig_type == 'bar':
        fig.update_layout(barmode = 'group')
        
    for i in range(N + 1, 1 + 3*N):
        fig["data"][i]['showlegend'] = False
    
    fig.update_layout(title = My_Title, title_x = 0.5, legend_title = scope)
    
    fig.show(renderer = 'browser')
    return

In [7]:
from plotly.validators.scatter.marker import SymbolValidator
def Get_Symbols_Sequence():
    raw_symbols = SymbolValidator().values
    symbol_stems = []
    open_symbols = []
    dot_symbols = []
    open_dot_symbols = []

    for i in range(0, len(raw_symbols), 12):
        name = raw_symbols[i + 2].replace('-dot', '')
        symbol_stems.append(name)
        open_symbols.append(name + '-open')
        dot_symbols.append(name + '-dot')
        open_dot_symbols.append(name + '-open-dot')

    dot_symbols = list(set(dot_symbols).intersection(set(raw_symbols)))
    open_dot_symbols = list(set(open_dot_symbols).intersection(set(raw_symbols)))
    symbols_sequence = symbol_stems + open_symbols + dot_symbols + open_dot_symbols
    
    return symbols_sequence

def Get_MyColors():
    MyColors = ['#0000FF', '#FF0000', '#228B22', '#AB63FA', '#FFA15A', '#19D3F3', 
                '#fc79c6', '#B6E880', '#feea52', '#fecbfc', '#778899', '#8B0000','#a9a9a9'] 
    return MyColors

In [8]:
def Scatter_Plot(df, scope, X, Y, Color, Size, Symbol, addline, hover_info, My_title, PerPop = False):
    symbols_sequence = Get_Symbols_Sequence()
    data_list = [data for data in df.columns.to_list() if 'Monthly' in data]
    scope_dict = {'State': 'State', 'Congressional District':'CDistrict', 'County':'County Name'}
    
    if PerPop:
        Y = Y + '/10000 pop'
        if Color in data_list:
            Color = Color + '/10000 pop'
        if Size in data_list:
            Size = Size + '/10000 pop'
    
    if Color in data_list:
        data_mean = df[Color].mean()
        data_min = df[Color].min()
        data_max = df[Color].max()
        loc_y = - 0.21
    else:
        data_mean = 1
        data_min = 1
        data_max = 1
        loc_y = - 0.11
    
    MyColors_Cont = 'Portland'
    
    if (scope == 'County') & (Color == scope_dict[scope]):
        MyColors_Disc = px.colors.qualitative.Alphabet
    else:
        MyColors_Disc = Get_MyColors()
    
    fig = px.scatter(df, x = X, y = Y, color = Color, size = Size, symbol = Symbol, 
                     symbol_sequence = symbols_sequence,
                     hover_data = hover_info, 
                     color_continuous_scale = MyColors_Cont, 
                     color_continuous_midpoint = data_mean, 
                     range_color = [data_min, data_max], 
                     color_discrete_sequence = MyColors_Disc, 
                     render_mode = 'svg')
    
    if addline & (scope == 'State'):
        cond1 = (X == 'Date') and ((Symbol != None) | (Color == 'Month') | (Color == 'Year'))
        cond2 = (X == 'Month') and (Symbol != 'Year') and (Color != 'Year')
        if not (cond1 | cond2):
            n = len(fig.data)
            if n == 1:
                fig.data[0].update(mode = 'markers+lines', line_color = 'grey', line_shape = 'spline')
            else:
                for i in range(n):
                    fig.data[i].update(mode = 'markers+lines', line_shape = 'spline')
        elif cond1:
            fig1 = fig
            fig2 = px.line(df, x = X, y = Y, line_shape = 'spline')
            fig2['data'][0]['line']['color'] = 'grey'
            fig = go.Figure(data = fig2.data + fig1.data, layout = fig1.layout)
        elif cond2:
            fig1 = fig
            fig2 = px.line(df, x = X, y = Y, color = 'Year')
            fig = go.Figure(data = fig2.data + fig1.data, layout = fig1.layout)
            
    if addline & (scope != 'State'):
        date_list = ['Month', 'Year']
        cond_pass = Color in scope_dict.values()
        cond1 = (X == 'Date') and ((Symbol in date_list) | (Color in date_list))
        cond2 = (X == 'Month') and (Symbol != 'Year') and (Color != 'Year')
        if cond_pass and not (cond1 | cond2):
            n = len(fig.data)
            for i in range(n):
                fig.data[i].update(mode = 'markers+lines', line_shape = 'spline')
        elif X == 'Date':
            fig1 = fig
            fig2 = px.line(df, x = X, y = Y, color = scope_dict[scope], color_discrete_sequence = MyColors_Disc, 
                           line_shape = 'spline', render_mode = 'svg')
            fig = go.Figure(data = fig2.data + fig1.data, layout = fig1.layout)
        elif X == 'Month':
            fig1 = fig
            fig2 = px.line(df, x = X, y = Y, color = scope_dict[scope], line_dash = 'Year', 
                           color_discrete_sequence = MyColors_Disc, 
                           line_shape = 'spline', render_mode = 'svg')
            fig = go.Figure(data = fig2.data + fig1.data, layout = fig1.layout) 

    fig.update_layout(title = My_title, title_x = 0.5,  title_font_color = 'royalblue',
                      plot_bgcolor = '#F0F8FF', 
                      coloraxis_colorbar = dict(yanchor = 'top', xanchor = 'left', 
                                                y = - 0.11, x = 0, orientation = 'h', 
                                                ticklabelposition = 'inside left'))
    
    if Size == None:
        fig.update_traces(marker = {'size': 12})
    else:
        fig.add_annotation(showarrow = False, xref = 'x domain', yref = 'y domain', x = - 0.02, y = loc_y, 
                           text = "marker size = " + Size, font_color = 'red')
        
    fig.show(renderer = 'browser')
    return

In [9]:
def Line_Plot(df, scope, X, Y, Color, Line_dash, Symbol, My_title, PerPop):
    symbols_sequence = Get_Symbols_Sequence()
    scope_dict = {'State': 'State', 'Congressional District':'CDistrict', 'County':'County Name'}
    if (scope == 'County') & (Color == scope_dict[scope]):
        MyColors_Disc = px.colors.qualitative.Alphabet
    else:
        MyColors_Disc = Get_MyColors()
    
    if Symbol == None:
        isMarkers = False
    else:
        isMarkers = True
        
    if (X == 'Month') & (Color != 'Year') & (Line_dash != 'Year') & (Symbol != 'Year') & (scope == 'State'):
        Color = 'Year'
        MyColors_Disc = ['#0000FF']
        
    if PerPop:
        Y = Y + '/10000 pop'
        
    fig = px.line(df, x = X, y = Y, color = Color, line_dash = Line_dash, 
                  markers = isMarkers, symbol = Symbol, 
                  color_discrete_sequence = MyColors_Disc, 
                  line_shape = 'spline', render_mode = 'svg',
                  symbol_sequence = symbols_sequence)
    
    fig.update_layout(title = My_title, title_x = 0.5,  title_font_color = 'royalblue', 
                      plot_bgcolor = '#F0F8FF')
    
    fig.show(renderer = 'browser')
    return

In [10]:
def Bar_Plot(df, scope, X, Y, Color, Pattern, hover_info, My_title, PerPop):
    MyColors_Disc = Get_MyColors()
    MyColors_Cont = 'Portland'
    Pattern_Sequence = ['', '/', '.', '+', '-', '\\', '|', 'x']
    data_list = [data for data in df.columns.to_list() if 'Monthly' in data]
    
    if PerPop:
        Y = Y + '/10000 pop'
        if Color in data_list:
            Color = Color + '/10000 pop'
    
    if Color in data_list:
        data_mean = df[Color].mean()
        data_min = df[Color].min()
        data_max = df[Color].max()
        Q1 = np.percentile(df[Color], 25, interpolation = 'midpoint')
        Q3 = np.percentile(df[Color], 75, interpolation = 'midpoint')
        IQR = Q3 - Q1
        upper = Q3 + 2.5*IQR
        lower = Q1 - 2.5*IQR
        if data_max > upper:
            Mask = df[Color] < upper
            data_mean = df[Mask][Color].mean()
            data_max = upper
    else:
        data_mean = 1
        data_min = 1
        data_max = 1

    fig = px.bar(df, x = X, y = Y, color = Color,
                 pattern_shape = Pattern, 
                 hover_data = hover_info,
                 color_discrete_sequence = MyColors_Disc, 
                 color_continuous_scale = MyColors_Cont, 
                 color_continuous_midpoint = data_mean, 
                 range_color = [data_min, data_max], 
                 pattern_shape_sequence = Pattern_Sequence)
    
    fig.update_layout(barmode = 'overlay')
    
    if (X == 'Month') | (scope != 'State'):
        fig.update_layout(barmode = 'group', bargap = 0.1, bargroupgap = 0)
        fig.update_traces(marker_line_width = 0)
    
    fig.update_layout(title = My_title, title_x = 0.5,  title_font_color = 'royalblue',
                      plot_bgcolor = '#F0F8FF', 
                      coloraxis_colorbar = dict(yanchor = 'top', xanchor = 'left', 
                                                y = - 0.11, x = 0, orientation = 'h', 
                                                ticklabelposition = 'inside left'))
    
    fig.show(renderer = 'browser')

In [11]:
def Box_Plot(df, X, Y, Color, PerPop):
    if PerPop:
        Y = Y + '/10000 pop'
    fig = px.box(df, x = X, y = Y, color = Color, 
                 color_discrete_sequence = Get_MyColors(), 
                 boxmode = 'overlay', points = 'suspectedoutliers')
    
    fig.show(renderer = 'browser')

In [12]:
def TimeSeries_VaccData_Plot(df, sIndex, Y, My_title):
    if sIndex == 'County Code':
        Color = 'County Name'
        MyColors_Disc = px.colors.qualitative.Alphabet
    else:
        Color = sIndex
        MyColors_Disc = Get_MyColors()
    
    fig = px.scatter(df, x = df.index.get_level_values(0), y = Y, color = Color, 
                     color_discrete_sequence = MyColors_Disc)
    
    fig.update_layout(title = My_title, title_x = 0.5,  title_font_color = 'royalblue', plot_bgcolor = '#F0F8FF')
    
    fig.show(renderer = 'browser')

In [13]:
def Trend_Plots(datasets_list, titles_list, My_Title, Waves_Info):
    MyColors = ['#1616A7', '#2CA02C', '#D62728', '#1F77B4', '#3366CC']
    
    nL = len(datasets_list)
    nc = 1 + int(np.ceil(nL/6))
    n_rows = int(np.ceil(nL/nc))
    n_cols = min(nL, nc)
    
    try:
        fig = make_subplots(rows = n_rows, cols = n_cols,
                            x_title = 'Report Date', y_title = 'Normalized Trend Values',
                            subplot_titles = titles_list,
                            horizontal_spacing = 0.05, vertical_spacing = 0.06)
    except ValueError:
        return
    
    for i in range(nL):
        nr = int(i/nc) + 1
        nl = i + 1 - nc*(nr - 1)
        trace = px.line(datasets_list[i], color_discrete_sequence = MyColors)["data"]
        for n in range(len(trace)):
            fig.add_trace(trace[n], row = nr, col = nl)
    
    last_date = datasets_list[0].index[-1]
    for i in range(len(Waves_Info)):
        Wave = 'Wave ' + str(i + 1)
        edate = Waves_Info[Wave]['end'][1]
        sdate = Waves_Info[Wave]['start'][1]
        if edate != 'NA':
            midpoint = sdate + (edate - sdate)/2
        else:
            midpoint = sdate + (last_date - sdate)/2
        fig.add_vline(x = edate, line_width = 3, line_color = "black")
        fig.add_annotation(x = midpoint, row = 'all', col = 'all', text = Wave, **{'y': 1.05}, 
                           showarrow = False, font = {'size': 14, 'color': '#1f77b4'})
        
    fig.update_layout(title = My_Title, title_x = 0.5, title_font_color = 'royalblue', title_font_size = 30,
                      showlegend = False, autosize = False, width = 1400, height = int(840*n_rows/n_cols), 
                      margin=dict(l = 80, r = 10, t = 100, b = 50), plot_bgcolor = '#F0F8FF')
    
    fig.show(renderer = 'browser')
    return

In [14]:
def Mcorr_HeatMap(Matrix, r_method, R):
    z = Matrix.corr(method = r_method)
    
    if r_method == 'pearson':
        pval = Matrix.corr(method = lambda x, y: pearsonr(x, y)[1]) - np.eye(*z.shape)
    elif r_method == 'spearman':
        pval = Matrix.corr(method = lambda x, y: spearmanr(x, y)[1]) - np.eye(*z.shape)
    else:
        print('Error: the R method should be either pearson or spearman')
        return
    
    p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))

    MyColors = px.colors.diverging.Portland
    
    if R == 'R squared':
        z = z**2
        cmin = 0
        MyTitle = 'R\u00b2 ' + r_method.capitalize() + ' Correlation Matrix'
    else:
        cmin = -1
        MyTitle = 'R ' + r_method.capitalize() + ' Correlation Matrix'
    
    fig = px.imshow(z, color_continuous_scale = MyColors, zmin = cmin, zmax = 1, text_auto = '.2f')

    fig.update_traces(customdata = np.moveaxis([z.round(4).astype(str) + p, pval], 0, -1),
                      hovertemplate = "%{customdata[0]}<br>Pvalue: %{customdata[1]:.4f}<extra></extra>")
    
    fig.update_layout(title = MyTitle, title_x = 0.5, title_font_color = 'royalblue', title_font_size = 30)
    
    fig.show(renderer = 'browser')
    return

In [15]:
def Mcorr_Scatter(Matrix, r_method):
    if r_method == 'spearman':
        df = Matrix.rank()
        MyTitle = 'Ranked Scatter Matrix'
    else:
        df = Matrix.copy()
        MyTitle = 'Scatter Matrix'
        
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', FutureWarning)
        fig = px.scatter_matrix(df, color_discrete_sequence = ['black'])
    fig.update_traces(diagonal_visible=False)
    fig.update_layout(title = MyTitle, title_x = 0.5, title_font_color = 'royalblue', title_font_size = 30)
    fig.show(renderer = 'browser')

#     g = sns.PairGrid(df)
#     g.map_diag(sns.distplot)
#     g.map_lower(sns.regplot)
#     g.map_upper(sns.kdeplot)
    
#     plt.show()

In [16]:
def Mp_corr_HeatMap(Matrix, R):
    z = pg.pcorr(Matrix)
    r_method = 'pearson'
    
    MyColors = px.colors.diverging.Portland
    
    if R == 'R squared':
        z = z**2
        cmin = 0
        MyTitle = 'R\u00b2 ' + r_method.capitalize() + ' Partial Correlation Matrix'
    else:
        cmin = -1
        MyTitle = 'R ' + r_method.capitalize() + ' Partial Correlation Matrix'
    
    fig = px.imshow(z, color_continuous_scale = MyColors, zmin = cmin, zmax = 1, text_auto = True)
    
    fig.update_layout(title = MyTitle, title_x = 0.5, title_font_color = 'royalblue', title_font_size = 30)
    
    fig.show(renderer = 'browser')
    return

In [17]:
def Pairwise_Corr_HeatMap(df, covar_list, r_method, R):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', FutureWarning)
        z = pg.pairwise_corr(df, covar = covar_list, method = r_method)

    z2 = z.copy()
    z2.Y = z.X
    z2.X = z.Y
    
    zcomb = pd.concat([z, z2])
    M = zcomb.pivot_table(index = 'X', columns = 'Y', values = 'r', aggfunc = np.sum, fill_value = 1, sort = True)
    pval = zcomb.pivot_table(index = 'X', columns = 'Y', values = 'p-unc', aggfunc = np.sum, fill_value = 1, sort = True)
    p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))

    MyColors = px.colors.diverging.Portland
    
    if R == 'R squared':
        M = M**2
        cmin = 0
        MyTitle = 'R\u00b2 ' + r_method.capitalize() + ' Partial Correlation Matrix'
    else:
        cmin = -1
        MyTitle = 'R ' + r_method.capitalize() + ' Partial Correlation Matrix'
        
    SubTitle = 'Given: '
    for c in covar_list:
        SubTitle += c + ', '    
    SubTitle = SubTitle[:-2]
    
    # ===================================================================================================
    def format_title(title, subtitle = None, subtitle_font_size = 14):
        title = f'<b>{title}</b>'
        if not subtitle:
            return title
        
        subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
        return f'{title}<br>{subtitle}'

    # ===================================================================================================
    
    fig = px.imshow(M, color_continuous_scale = MyColors, zmin = cmin, zmax = 1, text_auto = '.2f')

    fig.update_traces(customdata = np.moveaxis([M.round(4).astype(str) + p, pval], 0, -1),
                      hovertemplate = "%{customdata[0]}<br>Pvalue: %{customdata[1]:.4f}<extra></extra>")
    
    fig.update_layout(title = format_title(MyTitle, SubTitle), 
                      title_x = 0.5, title_y = 0.969, title_font_color = 'royalblue', 
                      title_font_size = 30, 
                      yaxis_title = None, xaxis_title = None)
    
    fig.update_layout(margin = dict(t = 80), paper_bgcolor="LightBlue")
    
    fig.show(renderer = 'browser')

    return